<h1 id="basics" style="font-family:verdana;"> 
    <center> Hybrid Recommendation System for Anime Movies!
    </center>
</h1>
<div style="width:100%;text-align: center;"> <img align=middle src="https://jw-webmagazine.com/wp-content/uploads/2020/04/chihiro042-1.jpg" alt="Recommendation" style="height:500px;margin-top:2rem;"> </div>



This data set contains information on user preference data from 73,516 users on 12,294 anime. Each user is able to add anime to their completed list and give it a rating and this data set is a compilation of those ratings.

## Main topics of the study can be seen below:

* [Aim of the study](#section-one)
* [Understanding the data](#section-two)
* [Preparation of data](#section-three)
* [User Based Collaborative Filtering](#section-four)
    * [Movies Watched by Random User](#section-five)
    * [The Other Users Who is Watching Same Movies](#section-six)
    * [Similarity Analysis](#section-seven)
    * [Score Calculation and Recommendation](#section-eight)
* [Item Based Collaborative Filtering](#section-nine)
    * [Movie Suggestions with Item Based Method](#section-ten)
    * [Suggested Movies](#section-eleven)
* [Conclusion](#section-twelve)
* [Keep in Touch!](#section-thirteen)


<a id="section-one"></a>
## 1. Aim of the Study

Recommendation systems are used to provide personalized recommendations to users, based on their preferences and past behavior. Two popular approaches to building recommendation systems are Item-based Collaborative Filtering (ICF) and User-based Collaborative Filtering (UCF).

Item-based Collaborative Filtering is a recommendation technique that looks at the similarity between items and recommends items that are similar to the ones the user has already liked. In contrast, User-based Collaborative Filtering is a recommendation technique that looks at the similarity between users and recommends items that similar users have liked in the past.

Hybrid Recommendation Systems combine these two approaches to provide better recommendations. By combining the strengths of both ICF and UCF, Hybrid systems can provide more accurate recommendations, especially when dealing with sparse data.

In Hybrid Recommendation Systems by Item and User-Based Collaborative Filtering, the system uses both the user's past behavior and the similarity between items to provide recommendations. It first identifies similar items to the ones the user has already liked and then identifies users who have similar tastes and preferences to the user. Finally, the system combines these two approaches to provide personalized recommendations to the user.

Overall, Hybrid Recommendation Systems by Item and User-Based Collaborative Filtering are becoming increasingly popular due to their ability to provide accurate and personalized recommendations, even when dealing with sparse data.

<div style="width:100%;text-align: center;"> <img align=middle src="https://149695847.v2.pressablecdn.com/wp-content/uploads/2021/10/image-97.png" alt="Hybrid" style="height:300px;margin-top:1rem;"> </div>

<a id="section-two"></a>
## 2. Understanding the Data

First of all we should import the libraries that will use during the analysis and recommendation parts.

In [1]:
# Lets import libraries

import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.width", 500)
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: '%.5f' % x)

In [2]:
# Lets import the dataset

movie = pd.read_csv(r"/kaggle/input/anime-recommendations-database/anime.csv")
rating = pd.read_csv(r"/kaggle/input/anime-recommendations-database/rating.csv")



In [3]:
# To understand the "check_df" functione can be used to decide the what should we do about the data.

datasets = [movie, rating]
def check_df(dataframe, head=10):
    print("########## First 10 Data #############")
    print(dataframe.head(head))
    print("########## Info #############")
    print(dataframe.info())
    print("########## Statistical Data #############")
    print(dataframe.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)
    print("########## Null Data #############")
    print(dataframe.isnull().sum())
    print("########## Variable Types #############")
    print(dataframe.dtypes)

for each in datasets:
    check_df(each)

########## First 10 Data #############
   anime_id                                               name                                              genre   type episodes  rating  members
0     32281                                     Kimi no Na wa.               Drama, Romance, School, Supernatural  Movie        1 9.37000   200630
1      5114                   Fullmetal Alchemist: Brotherhood  Action, Adventure, Drama, Fantasy, Magic, Mili...     TV       64 9.26000   793665
2     28977                                           Gintama°  Action, Comedy, Historical, Parody, Samurai, S...     TV       51 9.25000   114262
3      9253                                        Steins;Gate                                   Sci-Fi, Thriller     TV       24 9.17000   673572
4      9969                                      Gintama&#039;  Action, Comedy, Historical, Parody, Samurai, S...     TV       51 9.16000   151266
5     32935  Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...             Come

Before the start the analysis. Lets see the variables for each datasets;

1. Anime.csv

* anime_id - myanimelist.net's unique id identifying an anime.
* name - full name of anime.
* genre - comma separated list of genres for this anime.
* type - movie, TV, OVA, etc.
* episodes - how many episodes in this show. (1 if movie).
* rating - average rating out of 10 for this anime.
* members - number of community members that are in this anime's "group".

2. Rating.csv

* user_id - non identifiable randomly generated user id.
* anime_id - the anime that this user has rated.
* rating - rating out of 10 this user has assigned (-1 if the user watched it but didn't assign a rating).


<a id="section-three"></a>
## 3. Preparation of the Data

In this stage, If any null values are in the dataset, they will drop it from the data. 

In [4]:
# Movies that watched but not rated are dropped from the rating data.

watched_but_not_rated = rating.loc[rating["rating"] < 0,("user_id","anime_id")]
rating = rating[rating["rating"] >= 0]



In [5]:
movie.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37000,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26000,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25000,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17000,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16000,151266


In [6]:
# Before the start the analysis, datasets should be merged in a dataframe with "anime_id" variable

df = movie.merge(rating, how = "left", on = ["anime_id"])
df.head()

,anime_id,name,genre,type,episodes,rating_x,members,user_id,rating_y
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37000,200630,99.00000,5.00000
1,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37000,200630,152.00000,10.00000
2,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37000,200630,244.00000,10.00000
3,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37000,200630,271.00000,10.00000
4,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37000,200630,322.00000,10.00000


In [7]:
# After the filtering lets see the numerical variables analytics.

df.describe().T

,count,mean,std,min,25%,50%,75%,max
anime_id,6339607.00000,8908.83304,8887.26334,1.00000,1239.00000,6213.00000,14123.00000,34527.00000
rating_x,6339374.00000,7.67440,0.67093,1.67000,7.29000,7.70000,8.15000,10.00000
members,6339607.00000,184506.11148,190950.39618,5.00000,46781.00000,117091.00000,256325.00000,1013917.00000
user_id,6337239.00000,36747.91472,21013.40638,1.00000,18984.00000,36815.00000,54873.00000,73516.00000
rating_y,6337239.00000,7.80850,1.57250,1.00000,7.00000,8.00000,9.00000,10.00000


<a id="section-four"></a>
## 4. User Based Collaborative Filtering

Main aim of the study recommend the movies according to watched and rated movies by random user.

<div style="width:100%;text-align: center;"> <img align=middle src="https://dist.neo4j.com/wp-content/uploads/20170210203931/User-based-collaborative-filtering-algorithm.png" alt="User" style="height:300px;margin-top:1rem;"> </div>

In [8]:
# Lets see the total ratings of the movies according to dataset

rating_counts = pd.DataFrame(df["anime_id"].value_counts())
rating_counts

,anime_id
1535,34226
11757,26310
16498,25290
1575,24126
6547,23565
...,...
32823,1
31493,1
32844,1
23899,1


In [9]:
# In this study, less than 1000 rated movies are dropped from the data.

rare_movies = rating_counts[rating_counts["anime_id"] <= 1000].index
common_movie = df[~df["anime_id"].isin(rare_movies)]

In [10]:
# Lets prepare a Pivot Table between Users - Movies and Ratings

common_movie = common_movie.pivot_table(index = "user_id",
                         columns = "anime_id",
                         values = "rating_y")

common_movie.head()

anime_id,1,5,6,7,15,16,18,19,20,22,24,25,26,27,28,30,31,32,33,43,44,45,46,47,48,49,50,51,52,53,54,57,58,59,60,61,62,63,64,65,66,67,68,71,72,73,74,76,77,79,80,81,90,91,93,94,96,97,98,99,100,101,102,104,109,114,119,120,121,122,123,132,133,134,135,136,137,138,139,142,143,144,145,146,147,149,150,153,154,155,156,157,160,161,164,165,166,167,168,169,170,174,177,180,181,182,185,186,187,189,190,191,192,193,195,196,198,199,202,205,208,209,210,223,225,226,227,228,232,237,238,239,240,241,243,245,246,248,249,250,251,256,257,263,264,265,267,268,269,270,272,274,276,288,297,298,304,306,317,320,322,323,325,326,329,330,339,341,343,355,356,357,368,371,372,376,379,384,387,389,390,392,393,394,395,400,401,405,408,411,412,413,416,417,419,427,430,431,433,435,437,440,442,443,448,449,450,451,452,457,459,460,461,462,463,464,465,467,468,469,476,477,478,479,481,482,486,487,488,502,507,509,512,513,514,517,522,523,524,527,528,529,530,531,532,534,535,538,539,543,548,550,552,553,558,564,565,567,570,572,578,584,585,587,591,594,596,597,601,617,627,634,651,656,658,659,667,670,687,696,710,713,721,731,732,740,741,746,759,761,762,763,770,777,779,780,781,789,790,793,801,807,813,819,834,837,845,846,849,850,853,855,856,857,859,860,861,873,874,877,880,881,886,889,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,916,918,934,935,936,940,949,953,957,963,969,974,981,985,986,987,990,995,996,997,1002,1004,1015,1023,1029,1030,1033,1051,1066,1072,1074,1088,1117,1118,1119,1120,1121,1122,1132,1138,1140,1142,1172,1192,1195,1210,1212,1218,1222,1239,1240,1250,1254,1257,1281,1292,1303,1311,1313,1361,1363,1364,1365,1430,1462,1482,1498,1519,1526,1530,1531,1532,1535,1536,1546,1559,1562,1564,1565,1566,1568,1569,1571,1575,1579,1593,1594,1604,1606,1639,1668,1669,1686,1689,1690,1691,1696,1698,1699,1709,1719,1722,1723,1726,1731,1813,1815,1817,1818,1827,1829,1836,1840,1842,1860,1887,1889,1894,1915,1943,1953,1956,1965,2001,2002,2006,2025,2026,2034,2104,2107,2116,2129,2130,2131,2144,2148,2154,2159,2164,2167,2201,2216,2236,2246,2248,2251,2273,2369,2385,2386,2397,2398,2403,2404,2418,2472,2476,2494,2508,2581,2593,2596,2602,2605,2685,2759,2787,2795,2847,2889,2890,2899,2904,2923,2924,2926,2927,2951,2952,2961,2962,2963,2966,2969,2985,2986,2993,2994,3001,3002,3087,3089,3091,3092,3167,3210,3225,3226,3228,3229,3230,3231,3299,3328,3342,3354,3358,3359,3366,3375,3407,3420,3446,3455,3457,3467,3470,3503,3572,3588,3594,3613,3616,3652,3654,3655,3667,3702,3712,3713,3731,3759,3782,3783,3784,3785,3791,3848,3901,3927,3956,3958,3972,3974,4015,4026,4059,4063,4081,4087,4106,4107,4134,4151,4155,4163,4177,4181,4182,4186,4192,4196,4214,4224,4246,4280,4282,4382,4415,4437,4472,4477,4535,4548,4550,4551,4565,4581,4618,4654,4672,4719,4720,4722,4725,4726,4744,4752,4789,4793,4814,4835,4872,4884,4896,4898,4901,4918,4938,4970,4975,4981,4999,5028,5030,5034,5039,5040,5042,5060,5074,5079,5081,5112,5114,5118,5150,5152,5162,5177,5178,5204,5205,5258,5262,5277,5279,5300,5341,5342,5355,5356,5365,5391,5507,5521,5525,5530,5593,5630,5667,5671,5678,5680,5681,5682,5690,5717,5734,5764,5772,5781,5909,5940,5941,5955,5958,5978,6007,6023,6030,6033,6045,6046,6114,6127,6164,6166,6171,6178,6201,6205,6211,6213,6288,6324,6325,6347,6351,6372,6377,6381,6408,6421,6444,6500,6512,6547,6572,6573,6574,6594,6610,6634,6637,6645,6675,6676,6682,6702,6707,6746,6747,6772,6773,6791,6793,6802,6811,6862,6864,6880,6884,6895,6896,6922,6945,6954,6956,6974,6987,7017,7054,7058,7059,7079,7082,7088,7148,7193,7311,7337,7338,7465,7472,7590,7592,7593,7627,7647,7661,7662,7674,7711,7724,7739,7769,7785,7791,7805,7817,7858,8063,8074,8086,8100,8129,8142,8246,8247,8277,8311,8407,8408,8424,8425,8426,8440,8449,8460,8465,8476,8479,8514,8516,8525,8536,8557,8577,8630,8634,8675,8676,8740,8768,8769,8795,8841,8857,8861,8876,8888,8915,8917,8934,8937,9041,9047,9062,9065,9074,9117,9135,9136,9181,9201,9252,9253,9289,9314,9330,9331,9355,9366,9367,9379,9465,9471,9479,9510,9513,9515,9587,9617,9618,9656,9675,9712,9724,9734,9736,9750,9756,9760,9776,9790,9834,9863,9919,9922,99

<a id="section-five"></a>
### 4.1 Movies Watched by Random User


In [11]:
# This analysis will be done for the random user who selects from the "common_movie" dataframe.

random_user = int(pd.Series(common_movie.index).sample(1).values)

print("The Id of the user is: ", random_user)

The Id of the user is:  13183


In [12]:
# Lets see which movies watched by the random user.

random_user_df = common_movie[common_movie.index == random_user]

movies_watched = random_user_df.columns[random_user_df.notna().any()].tolist()

print(movies_watched)

[64, 65, 79, 93, 94, 226, 228, 855, 1292, 1535, 1575, 1594, 1719, 1887, 2904, 2993, 3092, 3457, 3713, 4087, 4214, 4752, 4814, 4898, 5060, 5530, 5680, 6707, 6746, 6895, 7059, 7791, 8074, 8424, 8479, 8769, 8795, 8841, 8888, 9065, 9253, 9926, 10790, 11123, 11285, 11617, 11757, 12079, 13659, 14289, 15085, 15451, 17513, 18041]


In [13]:
# Main dataframe that provided by Pivot Table is filtered by watched movies by Random User

movies_watched_df = common_movie[movies_watched]

movies_watched_df

anime_id,64,65,79,93,94,226,228,855,1292,1535,1575,1594,1719,1887,2904,2993,3092,3457,3713,4087,4214,4752,4814,4898,5060,5530,5680,6707,6746,6895,7059,7791,8074,8424,8479,8769,8795,8841,8888,9065,9253,9926,10790,11123,11285,11617,11757,12079,13659,14289,15085,15451,17513,18041
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.00000,10.00000,NaN,NaN,NaN,NaN,10.00000,NaN,NaN
2.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.00000,10.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.00000,NaN,NaN,NaN,3.00000,NaN,2.00000,NaN,NaN,NaN,NaN,2.00000,NaN,NaN,NaN,NaN,NaN,1.00000,NaN,7.00000,NaN,NaN,2.00000,2.00000,4.00000,NaN,1.00000,NaN,5.00000,NaN,NaN,9.00000,NaN,6.00000,NaN,NaN,2.00000,1.00000,NaN,1.00000,NaN,NaN,1.00000,NaN,NaN
7.00000,8.00000,7.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.00000,9.00000,NaN,7.00000,8.00000,9.00000,7.00000,NaN,NaN,NaN,NaN,7.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.00000,NaN,8.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.00000,8.00000,NaN,7.00000,NaN,NaN,8.00000,7.00000,7.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73512.00000,NaN,NaN,NaN,NaN,NaN,9.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73513.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73514.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# According to filtered data, lets see how many movies watched by other users.

user_movie_count = movies_watched_df.T.notnull().sum()
user_movie_count = user_movie_count.reset_index()
user_movie_count.columns = ["user_id", "movie_count"]

user_movie_count

,user_id,movie_count
0,1.00000,4
1,2.00000,0
2,3.00000,4
3,5.00000,17
4,7.00000,17
...,...,...
69318,73512.00000,1
69319,73513.00000,1
69320,73514.00000,0
69321,73515.00000,14


<a id="section-six"></a>
### 4.2 The Other Users Who is Watching Same Movies

In [15]:
# According to movie counts that obtained for the other users should be higher than the limit. 

# Limit mean is the %60-80 percentage of the total watched movies by random user. In this study this percentage is selected as %80.

users_same_movies = user_movie_count[user_movie_count["movie_count"] > 0.8*len(movies_watched)]

users_same_movies

,user_id,movie_count
7656,8115.00000,45
12412,13183.00000,54
16021,17033.00000,48
16082,17095.00000,45
23551,24988.00000,47
31679,33631.00000,48
37917,40273.00000,45
39431,41878.00000,49
40137,42635.00000,53
42994,45659.00000,49


In [16]:
# After the conditions that discussed above, final dataframe is prepared according to "users_same_movies" dataframe.

final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies["user_id"].values)],
                      random_user_df[movies_watched]])

final_df

anime_id,64,65,79,93,94,226,228,855,1292,1535,1575,1594,1719,1887,2904,2993,3092,3457,3713,4087,4214,4752,4814,4898,5060,5530,5680,6707,6746,6895,7059,7791,8074,8424,8479,8769,8795,8841,8888,9065,9253,9926,10790,11123,11285,11617,11757,12079,13659,14289,15085,15451,17513,18041
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8115.00000,6.00000,6.00000,7.00000,7.00000,7.00000,5.00000,7.00000,NaN,6.00000,10.00000,9.00000,6.00000,NaN,8.00000,9.00000,5.00000,NaN,7.00000,7.00000,7.00000,1.00000,7.00000,NaN,7.00000,7.00000,8.00000,8.00000,6.00000,8.00000,NaN,8.00000,8.00000,7.00000,5.00000,8.00000,9.00000,8.00000,7.00000,5.00000,NaN,9.00000,NaN,6.00000,NaN,7.00000,8.00000,7.00000,NaN,7.00000,7.00000,3.00000,8.00000,4.00000,8.00000
13183.00000,8.00000,9.00000,8.00000,7.00000,7.00000,9.00000,9.00000,8.00000,7.00000,8.00000,9.00000,9.00000,9.00000,8.00000,9.00000,8.00000,8.00000,9.00000,9.00000,8.00000,8.00000,9.00000,8.00000,8.00000,8.00000,8.00000,8.00000,8.00000,9.00000,9.00000,7.00000,8.00000,10.00000,8.00000,8.00000,8.00000,9.00000,8.00000,6.00000,9.00000,7.00000,8.00000,8.00000,8.00000,7.00000,9.00000,9.00000,7.00000,8.00000,9.00000,9.00000,8.00000,8.00000,7.00000
17033.00000,8.00000,8.00000,NaN,9.00000,9.00000,9.00000,9.00000,NaN,8.00000,10.00000,10.00000,9.00000,8.00000,10.00000,10.00000,8.00000,9.00000,9.00000,8.00000,8.00000,8.00000,9.00000,9.00000,9.00000,8.00000,8.00000,10.00000,9.00000,10.00000,9.00000,9.00000,10.00000,9.00000,7.00000,NaN,9.00000,7.00000,8.00000,9.00000,8.00000,10.00000,8.00000,8.00000,8.00000,9.00000,NaN,10.00000,NaN,9.00000,9.00000,6.00000,NaN,6.00000,8.00000
17095.00000,8.00000,8.00000,8.00000,NaN,NaN,8.00000,7.00000,7.00000,6.00000,8.00000,7.00000,7.00000,8.00000,8.00000,6.00000,5.00000,NaN,6.00000,7.00000,NaN,6.00000,5.00000,NaN,3.00000,NaN,7.00000,8.00000,1.00000,9.00000,7.00000,10.00000,8.00000,6.00000,6.00000,NaN,5.00000,8.00000,8.00000,NaN,6.00000,9.00000,3.00000,6.00000,4.00000,6.00000,7.00000,5.00000,5.00000,4.00000,4.00000,5.00000,6.00000,NaN,4.00000
24988.00000,10.00000,10.00000,10.00000,10.00000,10.00000,10.00000,10.00000,10.00000,1.00000,7.00000,10.00000,10.00000,10.00000,7.00000,9.00000,7.00000,10.00000,10.00000,10.00000,8.00000,7.00000,10.00000,10.00000,10.00000,10.00000,10.00000,4.00000,10.00000,9.00000,9.00000,9.00000,3.00000,10.00000,8.00000,8.00000,8.00000,5.00000,10.00000,8.00000,9.00000,8.00000,10.00000,7.00000,9.00000,NaN,NaN,8.00000,NaN,NaN,8.00000,7.00000,NaN,NaN,NaN
33631.00000,10.00000,10.00000,NaN,NaN,NaN,8.00000,10.00000,NaN,8.00000,10.00000,10.00000,9.00000,10.00000,9.00000,9.00000,9.00000,10.00000,10.00000,9.00000,NaN,9.00000,10.00000,10.00000,10.00000,10.00000,10.00000,10.00000,10.00000,10.00000,9.00000,10.00000,9.00000,8.00000,8.00000,10.00000,9.00000,10.00000,10.00000,8.00000,9.00000,9.00000,10.00000,10.00000,10.00000,10.00000,7.00000,9.00000,NaN,8.00000,10.00000,8.00000,7.00000,9.00000,10.00000
40273.00000,9.00000,10.00000,5.00000,7.00000,8.00000,10.00000,10.00000,NaN,6.00000,8.00000,10.00000,10.00000,9.00000,6.00000,10.00000,5.00000,NaN,8.00000,10.00000,5.00000,5.00000,9.00000,NaN,10.00000,6.00000,NaN,5.00000,10.00000,NaN,7.00000,8.00000,5.00000,9.00000,8.00000,5.00000,8.00000,NaN,9.00000,6.00000,7.00000,7.00000,NaN,9.00000,NaN,5.00000,8.00000,4.00000,8.00000,6.00000,NaN,4.00000,10.00000,1.00000,8.00000
41878.00000,7.00000,7.00000,5.00000,NaN,NaN,6.00000,7.00000,7.00000,6.00000,6.00000,7.00000,8.00000,7.00000,8.00000,7.00000,4.00000,8.00000,4.00000,7.00000,9.00000,4.00000,4.00000,8.00000,7.00000,9.00000,9.00000,8.00000,4.00000,10.00000,8.00000,4.00000,8.00000,4.00000,5.00000,9.00000,8.00000,9.00000,4.00000,7.00000,8.00000,10.00000,8.00000,4.00000,9.00000,4.00000,8.00000,5.00000,NaN,6.00000,7.00000,3.00000,8.00000,NaN,NaN
42635.00000,5.00000,5.00000,5.00000,7.00000,7.00000,8.00000,8.00000,8.00000,7.00000,9.00000,7.00000,7.00000,NaN,7.00000,7.00000,5.00000,8.00000,8.00000,7.00000,8.00000,5.00000,8.00000,7.00000,8.00000,7.00000,8.00000,7.00000,8.00000,7.00000,

In [17]:
# Lets see the correlation between the users who are located in final dataframe

corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()
corr_df = pd.DataFrame(corr_df, columns = ["corr"])
corr_df.index.names = ["user_id_1", "user_id_2"]
corr_df = corr_df.reset_index()

corr_df

,user_id_1,user_id_2,corr
0,24988.00000,57620.00000,-0.12851
1,17095.00000,42635.00000,-0.10905
2,24988.00000,17095.00000,-0.10705
3,33631.00000,45659.00000,-0.08989
4,13183.00000,33631.00000,-0.03364
...,...,...,...
113,8115.00000,17033.00000,0.62047
114,59643.00000,59406.00000,0.71430
115,59643.00000,17033.00000,0.74003
116,13183.00000,13183.00000,1.00000


<a id="section-seven"></a>
### 4.3 Similarity Analysis

In [18]:
# After the correlation, dataframe should be filtered by the correlation limit. This limit is choosen as 0.50 in this study.

top_users= corr_df[corr_df["corr"] > 0.50]
top_users.rename(columns= {"user_id_2": "user_id"}, inplace = True)

top_users

,user_id_1,user_id,corr
103,54713.00000,59406.00000,0.50233
104,40273.00000,59643.00000,0.50521
105,8115.00000,41878.00000,0.50981
106,59406.00000,17033.00000,0.51568
107,59643.00000,54713.00000,0.55310
108,17033.00000,51270.00000,0.55704
109,51270.00000,59406.00000,0.56507
110,59406.00000,8115.00000,0.56991
111,40273.00000,24988.00000,0.57624
112,8115.00000,59643.00000,0.60105


In [19]:
# Now, lets see the matched users according to anime_ids

top_users_rating = top_users.merge(rating[["user_id", "anime_id", "rating"]], how = "inner")
top_users_rating

,user_id_1,user_id,corr,anime_id,rating
0,54713.00000,59406.00000,0.50233,1,8
1,54713.00000,59406.00000,0.50233,5,8
2,54713.00000,59406.00000,0.50233,6,8
3,54713.00000,59406.00000,0.50233,8,7
4,54713.00000,59406.00000,0.50233,15,8
...,...,...,...,...,...
22205,13183.00000,13183.00000,1.00000,15085,9
22206,13183.00000,13183.00000,1.00000,15195,8
22207,13183.00000,13183.00000,1.00000,15451,8
22208,13183.00000,13183.00000,1.00000,17513,8


In [20]:
# To recommend the movies, we should consider both correlation and rating of the related movies help with "weighted_rating"

top_users_rating["weighted_rating"] = top_users_rating["corr"] * top_users_rating["rating"]

<a id="section-eight"></a>
### 4.4 Score Calculation and Recommendation

In [21]:
# Lets prepare a recommendation dataframe that aggregated by weighted_rating means.

recommendation_df = top_users_rating.groupby("anime_id").agg({"weighted_rating": "mean"})
recommendation_df = recommendation_df.reset_index()
recommendation_df

,anime_id,weighted_rating
0,1,5.04394
1,5,4.40316
2,6,4.83668
3,7,3.53112
4,8,3.65753
...,...,...
4194,33964,3.31878
4195,34103,4.53150
4196,34240,4.05621
4197,34324,2.76565


In [22]:
# To recommende the movies, I used the limit for the weighted_rating as 5.5 and sorted the dataframe to get the top movies.

movies_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 5.5].sort_values("weighted_rating", ascending = False)

In [23]:
# Letss see the anime names according to filtered data.

movies_to_be_recommend.merge(movie[["anime_id", "name"]])

,anime_id,weighted_rating,name
0,15195,8.00000,Hetalia: The Beautiful World
1,32281,6.25393,Kimi no Na wa.
2,15417,5.97264,Gintama&#039;: Enchousen
3,28675,5.94378,Kyoukai no Kanata Movie: I&#039;ll Be Here - M...
4,27821,5.89257,Fate/stay night: Unlimited Blade Works - Prologue
...,...,...,...
95,10087,5.52650,Fate/Zero
96,1029,5.51795,Omoide Poroporo
97,199,5.51732,Sen to Chihiro no Kamikakushi
98,1698,5.51340,Nodame Cantabile


In [24]:
# Here is the top 5 recommended animes!

top_5 = movie[movie["anime_id"].isin(movies_to_be_recommend["anime_id"])].head(5)

top_5

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37000,200630
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25000,114262
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16000,151266
6,11061,Hunter x Hunter (2011),"Action, Adventure, Shounen, Super Power",TV,148,9.13000,425855
8,15335,Gintama Movie: Kanketsu-hen - Yorozuya yo Eien...,"Action, Comedy, Historical, Parody, Samurai, S...",Movie,1,9.10000,72534


<div style="width:100%;text-align: center;"> <img align=middle src="https://images.wondershare.com/anireel/Resource/top-15-anime-movies-of-all-time-you-should-see-01.gif" alt="User" style="height:500px;margin-top:1rem;"> </div>

<a id="section-nine"></a>
## 5. Item Based Collaborative Filtering

<div style="width:100%;text-align: center;"> <img align=middle src="https://camo.githubusercontent.com/2c5912d076339dc02762e1c88772e6375d500b2ddd35b46c28ff7596be1ea0c1/68747470733a2f2f6d69726f2e6d656469756d2e636f6d2f6d61782f3830312f312a736b4b32667157694246377765485538536a75437a772e706e67" alt="User" style="height:500px;margin-top:1rem;"> </div>

<a id="section-ten"></a>
### 5.1 Movie Suggestions with Item Based Method

In [25]:
# Lets suggest a movie that rated as 10 by random user!

special_recommendation = rating[(rating['rating']==10) & (rating['user_id']==random_user)].reset_index().loc[0,'anime_id']


In [26]:
# Lets find the selected movie name
movie_name = movie.loc[movie['anime_id']==special_recommendation,'name'].to_list()
movie_name 

['Highschool of the Dead']

In [27]:
# Here is the selected movie's Id.

movie_name = common_movie[special_recommendation]
movie_name

user_id
1.00000       10.00000
2.00000            NaN
3.00000        6.00000
5.00000        2.00000
7.00000            NaN
                ...   
73512.00000        NaN
73513.00000        NaN
73514.00000        NaN
73515.00000    9.00000
73516.00000    9.00000
Name: 8074, Length: 69323, dtype: float64

In [28]:
# Lets find out the correlation according to selected movie name from the "common_movie" dataframe.

corr_movies = common_movie.corrwith(movie_name).sort_values(ascending=False).head(10)
corr_movies = corr_movies.reset_index()
corr_movies

,anime_id,0
0,8074,1.00000
1,9515,0.69322
2,9618,0.58136
3,8465,0.53925
4,6324,0.53842
5,6201,0.53084
6,11617,0.53050
7,5277,0.52946
8,2993,0.52740
9,5772,0.52590


<a id="section-eleven"></a>
### 5.2 Suggested Movies

In [29]:
# Here is the recommended 5 movies!

pd.DataFrame(corr_movies).iloc[1:,0].head(5).to_list()

[9515, 9618, 8465, 6324, 6201]

<a id="section-twelve"></a>
## 6. Conclusion

In conclusion, Hybrid Recommendation Systems by Item and User-Based Collaborative Filtering have become a popular approach to building recommendation systems due to their ability to provide accurate and personalized recommendations to users. By combining the strengths of both ICF and UCF, Hybrid systems can overcome the limitations of each approach and provide better recommendations, even when dealing with sparse data.

In Hybrid Recommendation Systems by Item and User-Based Collaborative Filtering, the system identifies similar items to the ones the user has already liked and then identifies users who have similar tastes and preferences to the user. By combining these two approaches, the system can provide more accurate and relevant recommendations to the user.

Overall, Hybrid Recommendation Systems by Item and User-Based Collaborative Filtering have been successful in many real-world applications, such as e-commerce, music, and movie recommendations. With the increasing amount of data generated by users, the importance of these systems will continue to grow in the future.

<a id="section-thireen"></a>
## 7. Keep in Touch!

You can follow my the other social media adresses to see this kind of works!

1. [GitHub](https://github.com/KeskinHakan)
2. [LinkedIn](https://www.linkedin.com/in/hakan-keskin-/)
3. [Medium](https://medium.com/@hakan-keskin)
